In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/Manish-k723/100-days-of-ML/main/train.csv')

In [3]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [4]:
X = df.drop(columns=['Survived'])
y = df['Survived']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)

In [6]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
30,1,male,40.0,0,0,27.7208,C
10,3,female,4.0,1,1,16.7000,S
873,3,male,47.0,0,0,9.0000,S
182,3,male,9.0,4,2,31.3875,S
876,3,male,20.0,0,0,9.8458,S


In [7]:
numerical_feature = ['Age','Fare']
numerical_transformer = Pipeline(steps = 
                                 [('imputer',SimpleImputer(strategy = 'mean')),
                                  ('scaler', StandardScaler())
                                  ])

categorical_feature = ['Sex','Embarked']
categorical_transformer = Pipeline(steps = 
                                   [('imputer',SimpleImputer(strategy = 'most_frequent')),
                                    ('ohe',OneHotEncoder(handle_unknown='ignore'))
                                   ])

In [9]:
preprocessor = ColumnTransformer([('num', numerical_transformer, numerical_feature),
                             ('cat',categorical_transformer, categorical_feature)
                             ],remainder = 'passthrough')

In [10]:
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])

In [11]:
from sklearn import set_config

set_config(display='diagram')
clf

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked'])])),
                ('classifier', LogisticRegression())])

In [12]:
param_grid = {
    'preprocessor__num__imputer__strategy': ['mean', 'median'],
    'preprocessor__cat__imputer__strategy': ['most_frequent', 'constant'],
    'classifier__C': [0.1, 1.0, 10, 100]
}

grid_search = GridSearchCV(clf, param_grid, cv=10)

In [14]:
grid_search.fit(X_train, y_train)

print(f"Best params:")
print(grid_search.best_params_)


Best params:
{'classifier__C': 0.1, 'preprocessor__cat__imputer__strategy': 'most_frequent', 'preprocessor__num__imputer__strategy': 'mean'}


In [15]:
print(f"Internal CV score: {grid_search.best_score_:.3f}")

Internal CV score: 0.812


In [16]:
import pandas as pd

cv_results = pd.DataFrame(grid_search.cv_results_)
cv_results = cv_results.sort_values("mean_test_score", ascending=False)
cv_results[['param_classifier__C','param_preprocessor__cat__imputer__strategy','param_preprocessor__num__imputer__strategy','mean_test_score']]

,param_classifier__C,param_preprocessor__cat__imputer__strategy,param_preprocessor__num__imputer__strategy,mean_test_score
0,0.1,most_frequent,mean,0.811718
1,0.1,most_frequent,median,0.811718
3,0.1,constant,median,0.811718
2,0.1,constant,mean,0.810309
4,1.0,most_frequent,mean,0.806084
5,1.0,most_frequent,median,0.806084
6,1.0,constant,mean,0.806084
7,1.0,constant,median,0.806084
8,10,most_frequent,mean,0.804675
10,10,constant,mean,0.803286
